### Imports

In [1]:
# Import all of the requried libraries
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
# import statistics
from statistics import mode,mean
from scipy import interpolate
import os
import json
import time

# These are all of the libraries that I manually created

# import IOfunctions as IO
# import GUIfunctions as GUI
# import Processfunctions as process

# Through 3D slicer
# start_index = 0 # starts at 195nm
# start_index = 742 # starts at 350nm
start_index = 790 # starts at 360nm
# start_index = 1070 # starts at 420nm



## Data Loading and Formatting 


#### Load, Format, and Save data

In [17]:
# This function combines loading the data with 
def loadDataset(dataPath,start_index=790,end_index=-1,sep=','):
    Dataset = []
    print("Loading in: ", dataPath)
    for name in os.listdir(dataPath):
        df = pd.read_csv(os.path.join(dataPath,name), sep=sep,engine='python', header=None)
        df = df.iloc[:, start_index:]
        data_arr = df.to_numpy()
        spectrum_arr = data_arr[1:, 1:]
        wavelength_arr = data_arr[0, 1:]
    return spectrum_arr,wavelength_arr

# LOADING DATASET 
FORMAT_DATASET = True
dataset_name = 'BreastPhantom_Nassir'
trialPath = "C:/Spectroscopy_TrackedTissueSensing/NassirDemo/data/Apr13/PatientA"
class0_name = 'Normal'
class1_name = 'Cancer'
dataPath0 = os.path.join(trialPath,class0_name)
dataPath1 = os.path.join(trialPath,class1_name)

# Load in the data
data_0, wavelength = loadDataset(dataPath0,start_index=start_index, sep=',')
labels0 = 0*np.ones(len(data_0))
data_1, wavelength = loadDataset(dataPath1,start_index=start_index, sep=',')
labels1 = 1*np.ones(len(data_1))

Loading in:  C:/Spectroscopy_TrackedTissueSensing/NassirDemo/data/Apr13/PatientA\Normal
Loading in:  C:/Spectroscopy_TrackedTissueSensing/NassirDemo/data/Apr13/PatientA\Cancer


## Visualization Trials

### Split data

In [18]:
print('Normal data shape: ',data_0.shape)
print('Cancer data shape: ',data_1.shape)
# print the labels shape
print('Normal labels: ',labels0.shape)
print('Cancer labels: ',labels1.shape)


Normal data shape:  (146, 2858)
Cancer data shape:  (124, 2858)
Normal labels:  (146,)
Cancer labels:  (124,)


### Display the raw data

In [20]:
# Displaying all of the spectra to visually inspect results

# This should be in GUI with all the inputs as parameters
wavelength_start = data_0[0,0,0]
wavelength_end = data_0[0,-1,0]

w = np.linspace(wavelength_start,wavelength_end,len(data_0[1]))
# # Display an example of data_0
GUI.plotSpectra(xdata=data_0[0,:,0],ydata=data_0[0,:,1],xlab='Wavelength(nm)',ylab='Reflected Intensity',
                title='Unprocessed data_0 Spectrum')
# Display an example of data_1
GUI.plotSpectra(xdata=data_1[0,:,0], ydata=data_1[0,:,1],xlab='Wavelength(nm)',ylab='Reflected Intensity',
                title='Unprocessed data_1 Spectrum' )



IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
# Function to plot all the spectra
def plotAll(data,title='', xtitle='', ytitle=''):
    plt.figure()
    for i in range(len(data)):
        plt.plot(data[i,:,0],data[i,:,1])
    plt.title(title)
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)


def plotWColourMap(data, title, xlabel, ylabel, step=1):
    """
    Plot all samples of class1 on a single figure using a colour map to denote chronological order.

    Args:
    - data (ndarray): a 3D array of shape (num_samples, num_wavelengths, 2) containing the spectral data
    - title (str): the title of the plot
    - xlabel (str): the label of the x-axis
    - ylabel (str): the label of the y-axis

    Returns:
    - None
    """

    # create the colour map
    cmap = plt.get_cmap('viridis')

    # plot the data with the colour map
    plt.figure()
    for i in range(len(data)-1):
        # plot every third spectra
        if i%step == 0:
            plt.scatter(data[i,:,0], data[i,:,1], s=0.1, c=cmap(i/len(data)))

    # add a color bar
    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm.set_array([])
    plt.colorbar(sm)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)


In [ ]:
# Plot all samples of class0 on a single figure without using plotSpectra
plotAll(data_0,'All normal Spectra','Wavelength[nm]','Reflected Intensity')

# Plot all samples of class1 on a single figure without using plotSpectra
plotAll(data_1,'All cancer Spectra','Wavelength[nm]','Reflected Intensity')

# GUI.plotSpectra(xdata=data_1[0,:,0], ydata=data_1[0,:,1],xlab='Wavelength(nm)',ylab='Reflected Intensity',
#                 title='Unprocessed data_1 Spectrum' )


In [ ]:
# PLOT the location of the max point for each spectrum
def plotMax(data, title, xlabel, ylabel):
    """
    Plot the location of the max point for each spectrum.

    Args:
    - data (ndarray): a 3D array of shape (num_samples, num_wavelengths, 2) containing the spectral data
    - title (str): the title of the plot
    - xlabel (str): the label of the x-axis
    - ylabel (str): the label of the y-axis

    Returns:
    - None
    """

    # find the max point for each spectrum
    max_points_indices = np.argmax(data[:,:,1], axis=1)
    # Find the corresponding wavelength
    max_points = data[:,max_points_indices,0]
    # plot the max points
    SampleNumber = range(len(data))
    plt.figure()
    plt.scatter(max_points[0],range(len(data)))
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    # change the range of the x-axis
    plt.xlim(400, 1000)

# concatinate the data_0 and data_1
d = np.concatenate((data_0, data_1), axis=0)
# Plot the location of the max point for each spectrum of class0
plotMax(d, 'Max point for all spectra', 'Wavelength index', 'Spectrum number')


In [ ]:
# data_0_save = data_0.copy()
# data_1_save = data_1.copy()

# data_0 = data_0_save.copy()
# data_1 = data_1_save.copy()


In [ ]:
# temporayily remove the peak caused by ambient light
data_0_new = data_0.copy()
data_1_new = data_1.copy()
# Set the spectra to zero between 600 and 650 nm
# This is to remove the background noise using no functions
def removeAmbientLight(data):
    start_index = 1110
    width = 20
    end_index = start_index + width
    for i in range(len(data)):
        data[i,start_index:end_index,1] = 0
    return data

data_0_new = removeAmbientLight(data_0_new)
data_1_new = removeAmbientLight(data_1_new)

# concatinate the data_0 and data_1
d = np.concatenate((data_0_new, data_1_new), axis=0)
# Plot the location of the max point for each spectrum of class0
plotMax(d, 'Max point for all spectra with ambient peak removed', 'Wavelength index', 'Spectrum number')

### Preprocessing of the data
* Normalize so peak is 1
* Crop to 360nm to 1024nm
* Divide by the broadband output

In [ ]:
# Normalize the data
data_0_norm = data_0_new.copy()
data_1_norm = data_1_new.copy()
data_0_norm = process.normalize(data_0_norm)
data_1_norm = process.normalize(data_1_norm)

# Plot them again
# Plot all samples of class0 on a single figure without using plotSpectra
plt.figure()
for i in range(len(data_0_norm)):
    plt.scatter(data_0_norm[i,:,0] ,data_0_norm[i,:,1],s=0.1)
plt.title('All normal Spectra (MinMax Normalized))')
plt.xlabel('Wavelength[nm]')
plt.ylabel('Reflected Intensity')

# Plot all samples of class1 on a single figure without using plotSpectra
plt.figure()
for i in range(len(data_1_norm)):
    plt.scatter(data_1_norm[i,:,0],data_1_norm[i,:,1],s=0.5)
plt.title('All cancer Spectra (MinMax Normalized)')
plt.xlabel('Wavelength[nm]')
plt.ylabel('Reflected Intensity')

# # plot one cancer and one normal spectrum on a scatter plot
# plt.figure()
# plt.scatter(data_0_norm[0,:,0],data_0_norm[0,:,1],s=0.5)
# plt.scatter(data_1_norm[0,:,0],data_1_norm[0,:,1],s=0.5)
# plt.title('One cancer and one normal spectrum [MinMax Normalized]')
# plt.xlabel('Wavelength[nm]')
# plt.ylabel('Reflected Intensity')
# plt.legend(['Normal','Cancer'])

In [ ]:
# # Plot each spectra using a color map to show how the spectra change over time
# # This is to see if there is any pattern in the spectra
# # use scatter plot to show the data points

# # create the colour map
# cmap = plt.get_cmap('viridis')

# # plot the data with the colour map
# plt.figure()
# for i in range(len(data_1_norm)-1):
#     plt.scatter(data_1_norm[i,:,0],data_1_norm[i,:,1],s=0.1,c=cmap(i/len(data_1_norm)))

# # add a color bar
# sm = plt.cm.ScalarMappable(cmap=cmap)
# sm.set_array([])
# plt.colorbar(sm)

# plt.title('All normal Spectra (MinMax Normalized)')
# plt.xlabel('Wavelength[nm]')
# plt.ylabel('Reflected Intensity')




In [ ]:
FLAG_Baseline = True
# Load in the baseline 
baseline = baseline_BrOut # --------------------------------- flag
baseline = process.normalize(baseline)[:,1]

data_0_norm = data_0_new[:,280:,:].copy()
data_1_norm = data_1_new[:,280:,:].copy()
baseline = baseline[280:].copy()
tFunc = baseline

def divTfuc(inputData,tFunc, flag):    
    outputData = inputData.copy()
    if FLAG_Baseline:
        # For each spectra
        for i in range (inputData[:,:,1].shape[0]):
            data = inputData[i,:,1]
            # Divide by the baseline transfer function
            outputData[i,:,1] = data / tFunc 
    outputData = process.normalize(outputData)
    return outputData
# call the function
data_0_norm_T = divTfuc(data_0_norm,tFunc, FLAG_Baseline)
data_1_norm_T = divTfuc(data_1_norm,tFunc, FLAG_Baseline)
# Display the arguemtns and output
freq = data_0_norm[0,:,0]

# Plot all samples of class0 on a single figure without using plotSpectra
# assuming data_1_norm_T is the input data array
plotWColourMap(data_0_norm_T, 'All Normal Spectra (Tfunc Norm)', 'Wavelength[nm]', 'Reflected Intensity', )

# Plot all samples of class1 on a single figure using a colour map to denote chronilogical order
# assuming data_1_norm_T is the input data array
plotWColourMap(data_1_norm_T, 'All cancer Spectra (Tfunc Norm)', 'Wavelength[nm]', 'Reflected Intensity')


# plot the average of the cancer spectra and the average of the normal spectra on a scatter plot
plt.figure()
plt.scatter(data_0_norm_T[:,:,0].mean(axis=0),data_0_norm_T[:,:,1].mean(axis=0),s=0.5)
plt.scatter(data_1_norm_T[:,:,0].mean(axis=0),data_1_norm_T[:,:,1].mean(axis=0),s=0.5)
plt.title('Average cancer and normal spectra [Divided by broad band]')
plt.xlabel('Wavelength[nm]')
plt.ylabel('Reflected Intensity')
plt.legend(['Normal','Cancer'])

In [ ]:
data_0_norm = data_0_norm_T
data_1_norm = data_1_norm_T

# # Plots of the normalized spectra
# GUI.plotSpectra(xdata=data_0_norm[0,:,0],ydata=data_0_norm[0,:,1],xlab='Wavelength(nm)',ylab='Reflected Intensity',
#                 title='Normalized data_0 Spectrum' )
# GUI.plotSpectra(xdata=data_1_norm[0,:,0],ydata=data_1_norm[0,:,1],xlab='Wavelength(nm)',ylab='Reflected Intensity',
#                 title='Normalized data_1 Spectrum' )

In [ ]:
# labels1
# # reindex the labels
# labels1 = labels1.reset_index(drop=True)
# # get the index of cancer
# # cancer_index = labels1[labels1['label'] == 1].index


## Preprocessing Pipeline

In [ ]:
Dataset_df
# Extract the data from the dataframe
data = np.array(Dataset_df['Data'].tolist())
#pritn the shape of the data
print('Dimensions of the data: ',np.shape(data))
# Normalize the data
data = process.normalize(data)
# Remove ambient light peak
data = removeAmbientLight(data)[:,280:,:] # I need to ravamp this function to use the data to remove the ambient light
# Divide by the baseline transfer function
processed_data = divTfuc(data,tFunc, FLAG_Baseline)
# Turn the processed data into a singe data column in a dataframe
data_df = pd.DataFrame()
for i in range(processed_data.shape[0]):
    new_row = {'Data_preprocessed':processed_data[i,:,:]}
    data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)
# Add the data to the dataframe
processedData_df = pd.concat([Dataset_df, data_df], axis=1)


## Trials

### PCA fitting

In [ ]:
from sklearn.decomposition import PCA
# Get the data from the dataframe
data = np.array(processedData_df['Data_preprocessed'].tolist())[:,:,1]
# Create the PCA object
pca = PCA(n_components=3)
# Fit the PCA object to the data
pca.fit(data)
# Transform the data
data_pca = pca.fit_transform(data)
# Create a dataframe with the PCA data
data_pca_df = pd.DataFrame(data_pca, columns=['PCA1','PCA2','PCA3'])
# Add the PCA data to the dataframe
data_pca_df = pd.concat([processedData_df, data_pca_df], axis=1)
data_pca_df

### PCA - Class separation

In [ ]:
# Plotting parameters
a = 5
figsize = (a,a)
fontLabel = 15
fontTitle = int(fontLabel*1.25)
# All scatter plot color options: https://matplotlib.org/3.1.0/gallery/color/named_colors.html

# 2D PCA: Cancer vs Normal
labels = [0,1]
colours = ['g','tab:orange']
plt.figure(figsize=figsize)
for label, colour in zip(labels,colours):
    plt.scatter(data_pca_df[data_pca_df['Label (numeric)'] == label]['PCA1'],data_pca_df[data_pca_df['Label (numeric)'] == label]['PCA2'],c = colour,s=25)
    # plt.scatter(data_pca_df[data_pca_df['Label (numeric)'] == 1]['PCA1'],data_pca_df[data_pca_df['Label (numeric)'] == 1]['PCA2'],c = colour,s=25)
plt.title('2D PCA: Cancer vs Normal', fontsize=fontTitle)
plt.xlabel('PCA1', fontsize=fontLabel)
plt.ylabel('PCA2', fontsize=fontLabel)
plt.legend(['Normal','Cancer'], fontsize=fontLabel)

# 3D PCA: Cancer vs Normal
labels = [0,1]
colours = ['g','tab:orange']
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='3d')
for label, colour in zip(labels,colours):
    ax.scatter(data_pca_df[data_pca_df['Label (numeric)'] == label]['PCA1'],data_pca_df[data_pca_df['Label (numeric)'] == label]['PCA2'],data_pca_df[data_pca_df['Label (numeric)'] == label]['PCA3'],c = colour,s=25)
    # plt.scatter(data_pca_df[data_pca_df['Label (numeric)'] == 1]['PCA1'],data_pca_df[data_pca_df['Label (numeric)'] == 1]['PCA2'],c = colour,s=25)
ax.set_title('3D PCA: Cancer vs Normal', fontsize=fontTitle)
ax.set_xlabel('PCA1', fontsize=fontLabel)
ax.set_ylabel('PCA2', fontsize=fontLabel)
ax.set_zlabel('PCA3', fontsize=fontLabel)
ax.legend(['Normal','Cancer'], fontsize=fontLabel)

# 2D PCA: all classes in the dataset
# get the unique labels
labels = np.unique(data_pca_df['Label'])
colours = ['tab:orange','tab:red','tab:pink','tab:purple','g']
plt.figure(figsize=figsize)
for label, colour in zip(labels,colours):
    print('here')
    plt.scatter(data_pca_df[data_pca_df['Label'] == label]['PCA1'],data_pca_df[data_pca_df['Label'] == label]['PCA2'],c=colour,s=25)
plt.title('2D PCA: all classes in the dataset', fontsize=fontTitle)
plt.xlabel('PCA1', fontsize=fontLabel)
plt.ylabel('PCA2', fontsize=fontLabel)
plt.legend(labels, fontsize=fontLabel)

# 3D PCA: all classes in the dataset
# get the unique labels
labels = np.unique(data_pca_df['Label'])
colours = ['tab:orange','tab:red','tab:pink','tab:purple','g']
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='3d')
for label, colour in zip(labels,colours):
    ax.scatter(data_pca_df[data_pca_df['Label'] == label]['PCA1'],data_pca_df[data_pca_df['Label'] == label]['PCA2'],data_pca_df[data_pca_df['Label'] == label]['PCA3'],c=colour,s=25)
ax.set_title('3D PCA: all classes in the dataset', fontsize=fontTitle)
ax.set_xlabel('PCA1', fontsize=fontLabel)
ax.set_ylabel('PCA2', fontsize=fontLabel)
ax.set_zlabel('PCA3', fontsize=fontLabel)
ax.legend(labels, fontsize=fontLabel)

### PCA: Sample separation

In [ ]:
# 2D PCA: Sample1 vs Sample2 vs Sample3
# get the unique labels
labels = np.unique(data_pca_df['SampleID'])
colours = ['tab:orange','tab:orange','tab:red','tab:red','tab:pink','tab:pink']
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
for label, colour in zip(labels,colours):
    ax.scatter(data_pca_df[data_pca_df['SampleID'] == label]['PCA1'],data_pca_df[data_pca_df['SampleID'] == label]['PCA2'],c=colour,s=50)
ax.set_title('2D PCA: Sample1 vs Sample2 vs Sample3', fontsize=fontTitle)
ax.set_xlabel('PCA1', fontsize=fontLabel)
ax.set_ylabel('PCA2', fontsize=fontLabel)
ax.legend(labels, fontsize=fontLabel)

# 3D PCA: Sample1 vs Sample2 vs Sample3
# get the unique labels
labels = np.unique(data_pca_df['SampleID'])
colours = ['tab:orange','tab:purple','tab:red','tab:pink','tab:blue','tab:green']
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='3d')
for label, colour in zip(labels,colours):
    ax.scatter(data_pca_df[data_pca_df['SampleID'] == label]['PCA1'],data_pca_df[data_pca_df['SampleID'] == label]['PCA2'],data_pca_df[data_pca_df['SampleID'] == label]['PCA3'],c=colour,s=50)
ax.set_title('3D PCA: Sample1 vs Sample2 vs Sample3', fontsize=fontTitle)
ax.set_xlabel('PCA1', fontsize=fontLabel)
ax.set_ylabel('PCA2', fontsize=fontLabel)
ax.set_zlabel('PCA3', fontsize=fontLabel)
ax.legend(labels, fontsize=fontLabel)


In [ ]:
# Perform PCA on the data to scale it down to 2 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# concatenate the data
data = np.concatenate((data_0_norm,data_1_norm),axis=0)
print(data.shape)

pca.fit(data[:,:,1])
data_0_pca = pca.transform(data_0_norm[:,:,1])
data_1_pca = pca.transform(data_1_norm[:,:,1])

# Plot the PCA data separated by cancer and normal
plot_title = 'PCA'
plt.figure()
plt.scatter(data_0_pca[:,0],data_0_pca[:,1],s=25)
plt.scatter(data_1_pca[:,0],data_1_pca[:,1],s=25)
plt.title(plot_title)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend(['Cancer','Normal'])
# save the plot as a vector graphic
plt.savefig('Experiments/PrelimKidney/'+ plot_title + '.svg')

# Plot the PCA data separated by sample number
plot_title = 'All_Data_PCA'
plt.figure()



### LDA fitting

In [ ]:
# Proceed with LDA on the data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Get the data from the dataframe
data = np.array(processedData_df['Data_preprocessed'].tolist())[:,:,1]
# Get the labels from the dataframe
labels = np.array(processedData_df['Label (numeric)'].tolist())

# Create the LDA object
lda = LinearDiscriminantAnalysis(n_components=1)
# Fit the LDA object to the data and labels
lda.fit(data, labels)
# Transform the data
data_lda = lda.transform(data)
print(data_lda.shape)
# Create a dataframe with the LDA data
data_lda_df = pd.DataFrame(data_lda, columns=['LDA1'])
# Add the LDA data to the dataframe
data_lda_df = pd.concat([processedData_df, data_lda_df], axis=1)
data_lda_df

### LDA: Class separation

In [ ]:
# 1D LDA: Cancer vs Normal
labels = [0,1]
colours = ['g','tab:orange']
plt.figure(figsize=[12,3])
for label, colour in zip(labels,colours):
    plt.scatter(data_lda_df[data_lda_df['Label (numeric)'] == label]['LDA1'],y = np.zeros_like(data_lda_df[data_lda_df['Label (numeric)'] == label]['LDA1']) + 0.,c = colour,s=25)
plt.title('1D LDA fitting: Cancer vs Normal', fontsize=fontTitle)
plt.xlabel('LDA', fontsize=fontLabel)
plt.legend(['Normal','Cancer'], fontsize=fontLabel)

# 1D LDA: All classes in the dataset
# get the unique labels
labels = np.unique(data_lda_df['Label'])
colours = ['tab:orange','tab:red','tab:pink','tab:purple','g']
plt.figure(figsize=[12,5])
for label, colour in zip(labels,colours):
    plt.scatter(data_lda_df[data_lda_df['Label'] == label]['LDA1'],y = np.zeros_like(data_lda_df[data_lda_df['Label'] == label]['LDA1']) + 0.,c = colour,s=25)
plt.title('1D LDA fitting: all classes in the dataset', fontsize=fontTitle)
plt.xlabel('LDA', fontsize=fontLabel)
plt.legend(labels, fontsize=fontLabel)

### LDA: Sample separation

In [ ]:
# 1D LDA: Sample1 vs Sample2 vs Sample3
SeparationIdx = 0
SeparationIncrement = 0.05
labels = np.unique(data_lda_df['SampleID'])
colours = ['tab:orange','tab:orange','tab:red','tab:red','tab:pink','tab:pink']
fig = plt.figure(figsize=[12,5])
ax = fig.add_subplot(111)
for label, colour in zip(labels,colours):
    x = data_lda_df[data_lda_df['SampleID'] == label]['LDA1']
    y = data_lda_df[data_lda_df['SampleID'] == label]['Label (numeric)']
    # Spread out the data for better visibility
    y = np.where(y==0,y+SeparationIdx,y-SeparationIdx)
    ax.scatter(x,y,c=colour,s=50)
    SeparationIdx = SeparationIdx + SeparationIncrement
ax.set_title('1D LDA: Sample1 vs Sample2 vs Sample3', fontsize=fontTitle)
ax.set_xlabel('LDA1', fontsize=fontLabel)
ax.set_ylabel('Label (numeric)', fontsize=fontLabel)
ax.legend(labels, fontsize=fontLabel)

# 1D LDA: Sample1 vs Sample2 vs Sample3
SeparationIdx = 0
labels = np.unique(data_lda_df['SampleID'])[1:] # remove the first label
print(labels)
colours = ['tab:orange','tab:orange','tab:red','tab:red','tab:pink','tab:pink'][1:] # remove the first label
print(colours)
fig = plt.figure(figsize=[12,5])
ax = fig.add_subplot(111)
for label, colour in zip(labels,colours):
    print(label, colour)
    x = data_lda_df[data_lda_df['SampleID'] == label]['LDA1']
    y = data_lda_df[data_lda_df['SampleID'] == label]['Label (numeric)']
    # if y is 0 add 0.1 else subtract 0.1
    y = np.where(y==0,y+SeparationIdx,y-SeparationIdx)
    ax.scatter(x,y,c=colour,s=50)
    SeparationIdx = SeparationIdx + SeparationIncrement
ax.set_title('1D LDA: Sample1 vs Sample2 vs Sample3', fontsize=fontTitle)
ax.set_xlabel('LDA1', fontsize=fontLabel)
ax.set_ylabel('Label (numeric)', fontsize=fontLabel)
ax.legend(labels, fontsize=fontLabel)

### t-SNE